Attemping to infer the parameters as spatial-temporal functions from low-fidelity PDE solution (KDE).

The PDE is assumed to have form:
$$
    u_t + \frac{\partial}{\partial x}\bigg[\mathcal{V}(t,x)u\bigg] = \frac{\partial}{\partial x}\bigg[\mathcal{D}(t,x)u_x\bigg]
$$ where $\mathcal{V}, \mathcal{D}$ are parameterized as separate NNs.

(06/15/2023) Method does not work well at capturing time intervals where the PDE solution changes rapidly. The method may capture the steady state solution well.

In [ ]:
from PINN.PhysicsInformedROPDF import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

In [ ]:
data_path = "../data/LinearOscillator/OU_Noise_Energy_PINN_Data.mat"
# create PINN
pinn = PhysicsInformedROPDF(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")

In [ ]:
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**9,
    epochs=60, batch_print=50
)

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(np.log10(info["pde_loss"]), lw=2, color="red");
plt.title("PDE Loss vs. Epoch");

plt.figure(2, figsize=(16, 8));
plt.plot(np.log10(info["data_loss"]), lw=2, color="blue");
plt.title("Data Loss vs. Epoch");

plt.figure(3, figsize=(16, 8));
plt.plot(np.log10(info["reg_loss"]), lw=2, color="purple");
plt.title("Regularity Loss vs. Epoch");

#plt.figure(4, figsize=(16, 8));
#plt.plot(np.log(info["init_loss"]), lw=2, color="purple");

#plt.figure(5, figsize=(16, 8));
#plt.plot(np.log(info["boundary_loss"]), lw=2, color="purple");

In [ ]:
query = cartesian_data(pinn.tgrid, pinn.xgrid)

D_pred = pinn.D_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
G_pred = pinn.G_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, D_pred);
plt.colorbar()
#plt.savefig("./fig/diffusion_nn.png", dpi=100)

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, G_pred);
plt.colorbar()
#plt.savefig("./fig/drift_nn.png", dpi=100)

In [ ]:
idx = 200
p_exact = pinn.raw_data["pmc"]

plt.plot(p_pred[idx, :], label="pred")
plt.plot(p_exact[idx, :], label="truth")
plt.legend()

In [ ]:
plt.contourf(p_pred)

In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), p_exact)

In [ ]:
xgrid = pinn.xgrid.detach().numpy()
dx = xgrid[1]-xgrid[0]
plt.figure(1);
fig, ax = plt.subplots(2, 2);
ax[0, 0].plot(xgrid, p_exact[0, :], label="KDE", lw=1.0, color="red");
ax[0, 0].plot(xgrid, p_pred[0, :], label="PINN", lw=1.0, color="blue");
ax[0, 0].set_title(r"$t = 0$");

ax[0, 1].plot(xgrid, p_exact[125, :], label="KDE", lw=1.0, color="red");
ax[0, 1].plot(xgrid, p_pred[125, :], label="PINN", lw=1.0, color="blue");
ax[0, 1].set_title(r"$t = 2.5$");

ax[1, 0].plot(xgrid, p_exact[250, :], label="KDE", lw=1.0, color="red");
ax[1, 0].plot(xgrid, p_pred[250, :], label="PINN", lw=1.0, color="blue");
ax[1, 0].set_title(r"$t = 5.0$");

ax[1, 1].plot(xgrid, p_exact[-1, :], label="KDE", lw=1.0, color="red");
ax[1, 1].plot(xgrid, p_pred[-1, :], label="PINN", lw=1.0, color="blue");
ax[1, 1].set_title(r"$t = 10.0$");

fig.set_figwidth(8);
fig.tight_layout(pad=0.5);

plt.savefig("./fig/PINN.png", dpi=100);

In [ ]:
plt.figure(2);
# relative error in L^2 from KDE benchmark
l2_rel_err = np.sum(((p_pred-p_exact)**2)*dx, 1) / np.sum((p_exact**2)*dx, 1)
tgrid = pinn.tgrid.detach().numpy()
plt.plot(tgrid, l2_rel_err, color="red", lw=1.5);
plt.grid(True);
plt.title(r"Relative error in $L^2$");
plt.xlabel(r"$t$"); 
plt.ylabel(r"$L^2(\mathbb{R})$ error");
plt.savefig("./fig/PINN_rel_l2_err.png", dpi=100);

## Learning a pure advection

To verify that our PINN implementation is working well, we test the PINN on the following problem:
$$
    \begin{cases}
        u_t + au_x = 0\\
        \lim_{|x|\rightarrow\infty}u(t, x) = 0\\
        u(0, x) = u_0(x)
    \end{cases}
$$ where $u_0 = \mathcal{N}(x|0, 1)$. 

In [ ]:
from PINN.PhysicsInformedROPDF import *
# Testing
import matplotlib.pyplot as plt
import torch
import numpy as np
import scipy

# set random seeds
np.random.seed(10)
torch.manual_seed(10);

import numpy as np    
import matplotlib.pyplot as plt

class LinearAdvection1D:
    # Matrix for LA1D 
    A=0
   # Initialization of constants 
    def __init__(self, c, x0, xN, N, deltaT,T):
        self.c = c 
        self.x0 = x0   
        self.xN = xN 
        self.N = N   
        self.deltaT = deltaT   
        self.T = T       
    # CFL number funct.   
    def CFL(self):
        deltaX= (self.xN - self.x0)/self.N
        return np.abs(self.c*self.deltaT/deltaX)
    # check CFL number <=1 or not.
    def checkCFL(self):
        if (self.CFL()<=1):
            flag=True
        else:
            flag=False
        return flag
    # Matrix assembly of LA1D   
    def upwindMatrixAssembly(self):
        alpha_min=min(self.CFL(),0)
        alpha_max=max(self.CFL(),0)
        a1=[alpha_max]*(self.N-1)
        a2=[1+alpha_min-alpha_max]*(self.N)
        a3=[-alpha_min]*(self.N-1)
        self.A=np.diag(a1, -1)+np.diag(a2, 0)+np.diag(a3, 1)

    # Solve u=Au0
    def step(self,u0):
        return np.matmul(self.A,u0) 

#############  
# Start of the code
###################

# constants  
N, x0, xN, deltaT, c, T = 1000, -10.,10., 0.01, 1.0, 10.0
# initialization of constants
LA1D = LinearAdvection1D(c, x0, xN, N, deltaT,T) 

# initial value
x = np.linspace(LA1D.x0,LA1D.xN,LA1D.N)
nx = len(x)
u0=(1/(np.sqrt(2*np.pi)))*np.exp(-0.5*((x+5)**2))

#plot of initial value    
plt.plot(x,u0,label="Initial value")
plt.ylabel('u')
plt.xlabel('x')
plt.legend()

# save solutions
nt = int(LA1D.T/LA1D.deltaT)
tgrid = np.linspace(0, T, nt)
u_sol = np.zeros([nt, nx])


# calculating solution if CFL<=1
if LA1D.checkCFL():
    print("CFL number is: ", LA1D.CFL())
    LA1D.upwindMatrixAssembly()
    for i in range(0, nt):
        u_sol[i, :] = u0
        u = LA1D.step(u0)
        u0 = u
else:
    print("CFL number is greater than 1. CFL: ", LA1D.CFL())

# ploting the last solution
plt.plot(x,u,label="Solution at t = {}".format(LA1D.T))
plt.legend()
plt.grid(linestyle='dotted')

# save data for testing
scipy.io.savemat("../data/LinearOscillator/Pure_Advection.mat", {
    "xgrid": x.reshape(1, -1),
    "tgrid": tgrid.reshape(1, -1),
    "pmc": u_sol
})

# save a smaller dataset
scipy.io.savemat("../data/LinearOscillator/Pure_Advection_small_data.mat", {
    "xgrid": x.reshape(1, -1)[:, 0:-1:5],
    "tgrid": tgrid.reshape(1, -1),
    "pmc": u_sol[:, 0:-1:5]
})

In [ ]:
# inverse problem with PINN
#data_path = "../data/LinearOscillator/Pure_Advection.mat"
# subsampled spatial training data
data_path = "../data/LinearOscillator/Pure_Advection_small_data.mat"
# create PINN
pinn = PhysicsInformedROPDF(indim=2, outdim=1, data_path=data_path, scheduler="ExponentialLR")
# testing training
info = train(
    pinn, pinn.optimizer, pinn.scheduler, batch_size=2**10,
    epochs=50, batch_print=50, mode="all"
)

In [ ]:
# visualize losses
plt.figure(1, figsize=(16, 8));
plt.plot(np.log10(info["pde_loss"]), lw=2, color="red");
plt.title("PDE Loss vs. Epoch");

plt.figure(2, figsize=(16, 8));
plt.plot(np.log10(info["data_loss"]), lw=2, color="blue");
plt.title("Data Loss vs. Epoch");

# plt.figure(3, figsize=(16, 8));
# plt.plot(np.log10(info["reg_loss"]), lw=2, color="purple");
# plt.title("Regularity Loss vs. Epoch");

In [ ]:
query = cartesian_data(pinn.tgrid, pinn.xgrid)

D_pred = pinn.D_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
G_pred = pinn.G_nn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()
p_pred = pinn(query).reshape(pinn.nx, pinn.nt).T.detach().numpy()

# grids
tgrid = pinn.tgrid.detach().numpy()
xgrid = pinn.xgrid.detach().numpy()

p_exact = pinn.raw_data["pmc"]
dx = pinn.dx.item()
dt = pinn.dt.item()

In [ ]:
plt.figure(1);
plt.contourf(xgrid, tgrid, G_pred);
plt.colorbar();

In [ ]:
plt.plot(xgrid, G_pred[0:-1:50, :].T);

In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), p_pred);

In [ ]:
plt.contourf(pinn.xgrid.detach().numpy(), pinn.tgrid.detach().numpy(), pinn.raw_data["pmc"]);

In [ ]:
plt.figure(2);
# relative error in L^2 from KDE benchmark
l2_rel_err = np.sum(((p_pred-p_exact)**2)*dx, 1) / np.sum((p_exact**2)*dx, 1)
tgrid = pinn.tgrid.detach().numpy()
plt.plot(tgrid, l2_rel_err, color="red", lw=1.5);
plt.grid(True);
plt.title(r"Relative error in $L^2$");
plt.xlabel(r"$t$"); 
plt.ylabel(r"$L^2(\mathbb{R})$ error");

In [ ]:
plt.figure(1);
plt.plot(xgrid, p_pred[0, :], xgrid, p_pred[-1, :], color="red");
plt.plot(xgrid, p_exact[0, :], xgrid, p_exact[-1, :], color="green");